# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
pivot = customer_products.pivot_table(values='Quantity', columns='CustomerID', 
                                      index='ProductName', aggfunc='sum').fillna(0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
custID = 33
similar_cust = list(distances[custID].sort_values(ascending=False)[1:].head().index)
similar_cust

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
merged = pd.merge(ranked_products, pd.DataFrame(pivot[custID]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recommendations = merged[merged['Purchased']==0].head()
recommendations

,ProductName,Quantity,Purchased
0,Butter - Unsalted,3,0.0
1,Wine - Ej Gallo Sierra Valley,3,0.0
3,Soup - Campbells Bean Medley,3,0.0
4,Wine - Blue Nun Qualitatswein,3,0.0
6,Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
recommendations = {}
customers = list(data['CustomerID'].unique())

for customer in customers:
    similar_cust = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(pivot[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
rec_df = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
rec_df.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
rec_df

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
5,83106,Cocoa Butter,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Cheese - Mix,Sauce - Hollandaise
6,11253,Juice - Lime,Tomatoes Tear Drop,Pomello,"Potatoes - Instant, Mashed",Table Cloth - 53x69 Colour
7,35107,Bread - French Baquette,Bandage - Flexible Neon,"Lentils - Red, Dry",Thermometer Digital,Wine - Hardys Bankside Shiraz
8,15088,Soup Knorr Chili With Beans,Soupfoamcont12oz 112con,Veal - Sweetbread,Cocktail Napkin Blue,Bread - Raisin Walnut Oval
9,26031,Cheese - Mozzarella,Bananas,Quiche Assorted,"Lemonade - Natural, 591 Ml",Coffee - Hazelnut Cream


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [13]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'correlation'))), 
                         index=pivot.columns, columns=pivot.columns)

In [14]:
recommendations = {}
customers = list(data['CustomerID'].unique())

for customer in customers:
    similar_cust = list(distances[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(pivot[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])
    
rec_df = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
rec_df.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
rec_df

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Fondant - Icing,Grenadine,Mangoes,Veal - Osso Bucco,Flour - Whole Wheat
2,40094,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Juice - V8 Splash,"Veal - Brisket, Provimi,bnls",Anchovy Paste - 56 G Tube
3,23548,Olives - Stuffed,Wine - Charddonnay Errazuriz,Assorted Desserts,Soup - Campbells Tomato Ravioli,Juice - Orange
4,78981,"Tart Shells - Sweet, 4",Sprouts - Baby Pea Tendrils,Artichokes - Jerusalem,Chocolate - Dark,"Appetizer - Mini Egg Roll, Shrimp"
5,83106,Pastry - Cheese Baked Scones,Bouq All Italian - Primerba,Onions - Vidalia,Bread - Italian Roll With Herbs,"Turnip - White, Organic"
6,11253,Sauce - Demi Glace,"Nut - Chestnuts, Whole","Beef - Tenderlion, Center Cut","Mushroom - Trumpet, Dry","Soup - Canadian Pea, Dry Mix"
7,35107,Fond - Neutral,Snapple Lemon Tea,Truffle Cups - Brown,"Veal - Brisket, Provimi,bnls",Cheese Cloth No 100
8,15088,Browning Caramel Glace,Vaccum Bag 10x13,Pastry - Choclate Baked,Cheese - Parmesan Cubes,Wine - Fume Blanc Fetzer
9,26031,Spinach - Baby,Foam Cup 6 Oz,General Purpose Trigger,Banana Turning,Sprouts - Baby Pea Tendrils
